# Purposes
- En este librillo pretendemos comparar RandomForest contra SMTS y la codificación que, SMTS, utiliza.

## Prerequisites

In [1]:
import os
os.chdir('../')

# Utility functions

In [2]:
from sklearn.ensemble import RandomForestClassifier
from SMTS import SMTS
from Datos import Datos
import ClassifierUtils
import pickle
import numpy as np

SEED = 1
PKL_DIR = 'pkl/'

data = Datos()

## Common data

In [3]:
# import CodificationUtils

# X = data.getWindowsOverStimulatedInterval(
#     mode={'nWindows': 1},
#     codificationFn=CodificationUtils.trendFn
# )

# with open(PKL_DIR + 'trend_data', 'wb') as file:
#     pickle.dump(X, file)

In [4]:
with open(PKL_DIR + 'trend_data', 'rb') as file:
    X = pickle.load(file)

In [5]:
X, relationWithSeries = X

In [6]:
X_train, X_test, y_train, y_test = ClassifierUtils.windowedTrainTestSplit(
    X, relationWithSeries, trainSize=0.7, seed=SEED, dropColumns=[])

y_train, y_test =\
    np.asarray(X_train.pop('class')), np.asarray(X_test.pop('class'))

## RandomForestClassifier

In [7]:
rf_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=5, random_state=SEED)
rf_clf.fit(X_train.drop('id', axis=1), y_train)

RandomForestClassifier(max_leaf_nodes=5, random_state=1)

In [8]:
rf_clf.score(X_test.drop('id', axis=1), y_test)

0.7057770335544535

In [9]:
rf_scores = ClassifierUtils.windowedCrossVal(
    rf_clf, X, relationWithSeries, cv=10, seed=SEED)

print('CVScore: {} +/- {}'.format(np.mean(rf_scores), np.std(rf_scores)))

CVScore: 0.7550994921687717 +/- 0.0986875871858381


## SMTS

In [13]:
smts_clf = SMTS(j_ins=100, n_symbols=5, random_state=SEED)
smts_clf.fit(X_train, y_train)

In [14]:
smts_clf.score(X_test, y_test)

0.8333333333333334

In [15]:
smts_scores = ClassifierUtils.windowedCrossVal(
    smts_clf,
    X,
    relationWithSeries,
    cv=10,
    dropColumns=['class'],
    customEstimator=True,
    seed=SEED
)

print('CVScore: {} +/- {}'.format(np.mean(smts_scores), np.std(smts_scores)))

CVScore: 0.8 +/- 0.13877773329774218


# Conclusions
- Observamos una mejora notable del algoritmo SMTS frente a RandomForest.
- Esto se debe a que, SMTS, utiliza una codificación simbólica que representa el tiempo implícitamente a través del primer RandomForest que utiliza y la frecuencia con la que los elementos de una clase caen en una u otra hoja de forma ordenada.
  - Esto da a SMTS una ventaja pues dispone de un mayor nº de patrones que poder identificar proporcionados de la forma de un vector de frecuencias ordenado en función de los índices de los nodos hoja de cada árbol.
- La diferencia fundamental de RF y SMTS es:
  - La codificación simbólica
    - Mientras que RF barajea todas las muestras de todas las series, SMTS, les da un orden (en función de los índices de las hojas de un RandomForest que utiliza para generar el conjunto de datos simbólico) y registra un vector de frecuencias que representa a cada serie. -> Orden (gracias a la columna temporal empleada en el 1er RF) + BoW (vectores de frecuencias de aparición de determinados patrones de la ventana de una serie en concreto para dárselo a entrenar al 2º RF)
  - SMTS utiliza 2 RF:
    - El primero para generar el conjunto de datos simbólico introduciendo el tiempo como un atributo más.
    - El segundo para clasificar nuevos datos que lleguen (a los cuales les debe aplicar también una codificación simbólica basada en el primer RandomForest para no sesgar al modelo).